In [ ]:
!python -m pip install dask-cuda
!python -m pip install jupyter-server-proxy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.6/126.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2023.8.1
    Uninstalling dask-2023.8.1:
      Successfully uninstalled dask-2023.8.1
  Attempting uninstall: distributed
    Found existing installation: distributed 2023.8.1
    Uninstalling distributed-2023.8.1:
      Successfully uninstalled distributed-2023.8.1


In [ ]:
import gc
import time
import math
import numpy as np
import cupy as cp
import dask
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCUDACluster()
client = Client(cluster)

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:38381
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35577'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:44001', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44001
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48242
INFO:distributed.scheduler:Receive client connection: Client-4a330258-02bd-11ef-80ea-0242ac1c000c
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48254


In [ ]:
jaro_winkler_code = r"""
extern "C"{

__device__ float jaro_winkler(const char *str1,
                              const int len1,
                              bool *hash_str1,
                              const char *str2,
                              const int len2,
                              bool *hash_str2) {

    // Description: Computes the Jaro-Winkler distance between two strings

    // Inputs:
    // - str1: First string
    // - len1: Length of str1
    // - hash_str1: Working memory to keep track of which characters in str1 are
    //              matching to corresponding characters in str2
    // - str2: Second string
    // - len2: Length of str2
    // - hash_str2: Working memory to keep track of which characters in str2 are
    //              matching to corresponding characters in str1

    // Output:
    // - dist: Jaro-Winkler distance between str1 and str2

    // If either string is null, the Jaro-Winkler distance between str1 and str2 is 0

    if (len1 == 0 || len2 == 0) {

        return 0.0;

    } else {

        // We compute the number of matching characters between str1 and str2

        // We consider the characters max(len1, len2) / 2 - 1 away from each other

        int max_dist = max(len1, len2) / 2 - 1;

        int match = 0;

        for (int i = 0; i < len1; i++) {

            for (int j = max(0, i - max_dist); j < min(len2, i + max_dist + 1); j++) {

                if (str1[i] == str2[j] && hash_str2[j] == false) {

                    // Two characters are matching if they appear in both strings
                    // at most max_dist characters away from each other

                    hash_str1[i] = true;
                    hash_str2[j] = true;
                    match++;
                    break;

                }

            }

        }

        // If there is no matching characters between both strings,
        // the Jaro-Winkler distance between them is 0

        if (match == 0) {

            return 0.0;

        } else {

            // If a positive number of matching characters is found, we need to
            // compute the number of transpositions, that is, the number of matching
            // characters that are not in the right order divided by two

            float t = 0;

            int point = 0;

            for (int i = 0; i < len1; i++) {

                if (hash_str1[i] == true) {

                    while (hash_str2[point] == false) {

                        point++;

                    }

                    if (str1[i] != str2[point]) {

                        t++;

                    }

                    point++;

                }

            }

            t /= 2;

            // The Jaro distance between str1 and str2 is defined as follows:

            float dist;

            dist = (((float)match / (float)len1) + ((float)match / (float)len2) + (((float)match - t) / (float)match)) / 3.0;

            // To go from the Jaro distance to the Jaro-Winkler distance, we need
            // to compute the length of the common prefix between both strings

            int prefix = 0;

            for (int i = 0; i < min(min(len1, len2), 4); i++) {

                if (str1[i] == str2[i]) {

                    prefix++;

                } else {

                    break;

                }

            }

            // To obtain the Jaro-Winkler distance, we adjust the Jaro distance
            // for the length of the common prefix between both strings

            dist += 0.1 * prefix * (1 - dist);

            return dist;

        }

    }

}

__global__ void jaro_winkler_kernel(char *str1,
                                    int *offsets1,
                                    bool *buffer1,
                                    int n1,
                                    char *str2,
                                    int *offsets2,
                                    bool *buffer2,
                                    int n2,
                                    float *output) {

    // Inputs:
    // - str1: First vector of strings stored as an arrow (i.e., concatenated
    //         next to each other)
    // - offsets1: Vector storing the index where each string in str1 starts
    // - buffer1: Working memory to keep track of which characters in str1 are
    //            matching to corresponding characters in str2
    // - n1: Number of strings contained in str1
    // - str2: Second vector of strings stored as an arrow
    // - offsets2: Vector storing the index where each string in str2 starts
    // - buffer2: Working memory to keep track of which characters in str2 are
    //            matching to corresponding characters in str1
    // - n2: Number of strings contained in str2
    // - output: Vector storing the computed Jaro-Winkler distances

    const int id = threadIdx.x + blockDim.x * blockIdx.x;

    const int idx = id / n2; // Index of the string considered in str1

    const int idy = id % n2; // Index of the string considered in str2

    if (idx < n1 && idy < n2) {

        // Move the pointer to the first character of the string we are considering

        char *string1 = str1 + offsets1[idx];

        // Computing the length of the string we are considering

        int len1 = offsets1[idx + 1] - offsets1[idx];

        // Move the pointer to the first element of the working memory

        bool *hash_str1 = buffer1 + idy * offsets1[n1] + offsets1[idx];

        char *string2 = str2 + offsets2[idy];

        int len2 = offsets2[idy + 1] - offsets2[idy];

        bool *hash_str2 = buffer2 + idx * offsets2[n2] + offsets2[idy];

        // Compute the Jaro-Winkler Distance between str1[idx] and str2[idy]

        output[id] = jaro_winkler(string1, len1, hash_str1, string2, len2, hash_str2);

    }

}

}
"""

In [ ]:
jaro_winkler_kernel = cp.RawKernel(jaro_winkler_code, 'jaro_winkler_kernel')

In [ ]:
mempool = cp.get_default_memory_pool()

In [ ]:
def jaro_winkler_gpu(str1, str2):

    mempool = cp.get_default_memory_pool()

    # Description: Computes the Jaro-Winkler distance between all pairs of strings in str1 and str2

    # Inputs:
    # - str1: first list of strings
    # - str2: second list of strings

    # Output: Matrix of dimension len(str1) x len(str2), with the (i,j)th element
    #         containing the Jaro-Winkler distance between str1[i] and str2[j]

    n1 = len(str1) # Number of strings contained in str1

    # Storing strings contained in str1 as an arrow (i.e., concatenated next to each other)

    str1_arrow = np.array(list(''.join(str1)), dtype = 'S1').view(np.int8)

    str1_arrow_gpu = cp.array(str1_arrow)

    # Vector storing where each string starts and ends: str1[i] begins at offsets[i]
    # and ends at offsets[i + 1] - 1 (inclusively)

    offsets1 = np.append([0], np.cumsum([len(row) for row in str1])).astype(np.int32)

    offsets1_gpu = cp.array(offsets1)

    n2 = len(str2)

    str2_arrow = np.array(list(''.join(str2)), dtype = 'S1').view(np.int8)

    str2_arrow_gpu = cp.array(str2_arrow)

    offsets2 = np.append([0], np.cumsum([len(row) for row in str2])).astype(np.int32)

    offsets2_gpu = cp.array(offsets2)

    buffer1 = cp.zeros(offsets1[n1] * n2, dtype = bool)

    buffer2 = cp.zeros(offsets2[n2] * n1, dtype = bool)

    output_gpu = cp.zeros(n1 * n2, dtype = cp.float32) # Create output vector

    NUM_THREADS = 1024 # Threads per Block

    NUM_BLOCKS = math.ceil(n1 * n2 / 1024) # Blocks per Grid

    # Call GPU Kernel

    jaro_winkler_kernel((NUM_BLOCKS,), (NUM_THREADS,), (str1_arrow_gpu, offsets1_gpu, buffer1, n1, str2_arrow_gpu, offsets2_gpu, buffer2, n2, output_gpu))

    # Transfer output to CPU

    output = cp.asnumpy(output_gpu)

    # Memory management: clean GPU memory

    del str1_arrow, offsets1, buffer1, str2_arrow, offsets2, buffer2, str1_arrow_gpu, offsets1_gpu, str2_arrow_gpu, offsets2_gpu, output_gpu

    mempool.free_all_blocks()

    return output.reshape((n1, n2))

In [ ]:
def jaro_winkler(str1, str2):

  # Computes the Jaro-Winkler similarity between str1 and str2

  len1 = len(str1)

  hash_str1 = [False] * len1

  len2 = len(str2)

  hash_str2 = [False] * len2

  if (len1 == 0 or len2 == 0):

    return 0

  else:

    # We compute the number of matching characters between str1 and str2

    # We consider the characters max(len1, len2) / 2 - 1 away from each other

    max_dist = max(len1, len2) // 2 - 1

    match = 0

    for i in range(len1):

      for j in range(max(0, i - max_dist), min(len2, i + max_dist + 1)):

        if (str1[i] == str2[j] and not hash_str2[j]):

          # Two characters are matching if they appear in both strings
          # at most max_dist characters away from each other

          hash_str1[i] = True
          hash_str2[j] = True
          match += 1
          break

    # If there is no matching characters between both strings,
    # the Jaro-Winkler distance between them is 0

    if match == 0:

      return 0

    else:

      # If a positive number of matching characters is found, we need to
      # compute the number of transpositions, that is, the number of matching
      # characters that are not in the right order divided by two

      t = 0

      point = 0

      for i in range(len1):

        if hash_str1[i]:

          while (not hash_str2[point]):

            point += 1

          if (str1[i] != str2[point]):

            t += 1

          point += 1

          t /= 2

        # The Jaro distance between str1 and str2 is defined as follows:

      dist = ((match / len1) + (match / len2) + ((match - t) / match)) / 3

      # To go from the Jaro distance to the Jaro-Winkler distance, we need
      # to compute the length of the common prefix between both strings

      prefix = 0

    for i in range(min(min(len1, len2), 4)):

      if (str1[i] == str2[i]):

        prefix += 1

      else:

        break

    # To obtain the Jaro-Winkler distance, we adjust the Jaro distance
    # for the length of the common prefix between both strings

    dist += 0.1 * prefix * (1 - dist)

    return dist

In [ ]:
def jaro_winkler_cpu(str1, str2):

  return np.array([[jaro_winkler(x, y) for y in str2] for x in str1])

In [ ]:
array_names = da.from_array(['Jacob', 'Yasmeen', 'Myriam', 'Isabelle', 'Hamida'] * 6000, chunks=(10000,))

In [ ]:
mempool.free_all_blocks()

In [ ]:
dask.config.set({"optimization.fuse.active": True})

with dask.annotate(resources={'GPU': 1}):
    jw_matrix = da.blockwise(jaro_winkler_gpu, 'ij', array_names, 'i', array_names, 'j', dtype = np.float32)

indices = da.argwhere(jw_matrix >= 0.75)

/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:2089: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  ind = stack([ind[i].ravel() for i in range(len(ind))], axis=1)


In [ ]:
%%time
indices.compute()

In [ ]:
gc.collect()

20

In [ ]:
cluster = LocalCluster()
client = client = cluster.get_client()

/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34409 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:41337
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:34409/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42861'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41745'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:33421', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:33421
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:34006
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36633', name: 0, status: init, memory:

In [ ]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34409/status,
Dashboard: http://127.0.0.1:34409/status,Workers: 2
Total threads: 2,Total memory: 12.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41337,Workers: 2
Dashboard: http://127.0.0.1:34409/status,Total threads: 2
Started: Just now,Total memory: 12.67 GiB
Comm: tcp://127.0.0.1:36633,Total threads: 1
Dashboard: http://127.0.0.1:37599/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:42861,


In [ ]:
array_names = da.from_array(['Jacob', 'Yasmeen', 'Myriam', 'Isabelle', 'Hamida'] * 4000, chunks=(1000,))

jw_matrix = da.blockwise(jaro_winkler_cpu, 'ij', array_names, 'i', array_names, 'j', dtype = np.float32)

indices = da.argwhere(jw_matrix >= 0.75)

<ipython-input-24-0d631f45ee25>:3: PerformanceWarning: Increasing number of chunks by factor of 20
  jw_matrix = da.blockwise(jaro_winkler_cpu, 'ij', array_names, 'i', array_names, 'j', dtype = np.float32)


In [ ]:
%%time
indices.compute()

INFO:distributed.core:Connection to tcp://127.0.0.1:34018 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:36633', name: 0, status: running, memory: 419, processing: 2> (stimulus_id='handle-worker-cleanup-1714017763.570745')
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-62547' coro=<Client._gather.<locals>.wait() done, defined at /usr/local/lib/python3.10/dist-packages/distributed/client.py:2208> exception=AllExit()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/distributed/client.py", line 2217, in wait
    raise AllExit()
distributed.client.AllExit
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-62539' coro=<Client._gather.<locals>.wait() done, defined at /usr/local/lib/python3.10/dist-packages/distributed/client.py:2208> exception=AllExit()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/distributed/cl

KeyboardInterrupt: 

# Estimation


In [ ]:
!pip install -U tensorly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import tensorly as tl
import scipy
import numpy as np
import random

In [ ]:
values = [[i,j,k] for i in range(3) for j in range(3) for k in range(3)]

unique_values = np.stack([np.eye(3)[list] for list in values])

unique_values

array([[[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.]],

       [[1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.]],

       [[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]],

       [[1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]],

       [[1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.]],

       [[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.]],

       [[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.]],

       [[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]],

       [[0., 1., 0.],
        [0., 1., 0.],
    

In [ ]:
pi_1 = [0.1, 0.4, 0.5]
pi_0 = [0.5, 0.4, 0.1]
lmbda = 0.2

counts = [lmbda * pi_1[i] * pi_1[j] * pi_1[k] + (1 - lmbda) * pi_0[i] * pi_0[j] * pi_0[k] for i in range(3) for j in range(3) for k in range(3)]

In [ ]:
estimation(tl.tensor(unique_values), tl.tensor(counts))

([array([0.2, 0.8]), array([0.2, 0.8]), array([0.2, 0.8])],
 [array([[0.1, 0.5],
         [0.4, 0.4],
         [0.5, 0.1]]),
  array([[0.1, 0.5],
         [0.4, 0.4],
         [0.5, 0.1]]),
  array([[0.1, 0.5],
         [0.4, 0.4],
         [0.5, 0.1]])])

In [ ]:
def moment_pairs(unique_values, counts, dim1, dim2):

    # Output:
    # - moment: A two-dimensional tensor, each dimension representing the modalities taken by the variables dim1 and dim2, respectively

    moment = tl.tenalg.batched_outer([unique_values[:,dim1,:], unique_values[:,dim2,:]])

    return tl.tenalg.mode_dot(moment, counts, mode = 0) / np.sum(counts)

In [ ]:
E_1_2 = moment_pairs(tl.tensor(unique_values), tl.tensor(counts), 0, 1)

In [ ]:
E_1_2

array([[0.00109, 0.00381, 0.0141 ],
       [0.00381, 0.01629, 0.0909 ],
       [0.0141 , 0.0909 , 0.765  ]])

In [ ]:
np.linalg.pinv(tl.to_numpy(E_1_2))

array([[ 21.11589342,  53.81124334,  -6.81475639],
       [ 53.81124334, 137.13785704, -17.27476162],
       [ -6.81475639, -17.27476162,   3.48456746]])

In [ ]:
def symmetrization(unique_values, counts, dim1, dim2, dim3):

    truncated_tensor = unique_values[:,(dim1, dim2, dim3),:]

    E_1_2 = moment_pairs(unique_values, counts, dim1, dim2)

    E_1_3 = moment_pairs(unique_values, counts, dim1, dim3)

    E_2_3 = moment_pairs(unique_values, counts, dim2, dim3)

    trans_1 = tl.matmul(tl.transpose(E_2_3),
                        tl.tensor(np.linalg.pinv(tl.to_numpy(E_1_2))))

    trans_2 = tl.matmul(tl.transpose(E_1_3),
                        tl.tensor(np.linalg.pinv(np.transpose(tl.to_numpy(E_1_2)))))

    truncated_tensor[:,0,:] = tl.tenalg.mode_dot(trans_1, truncated_tensor[:,0,:], 0)

    truncated_tensor[:,1,:] = tl.tenalg.mode_dot(trans_2, truncated_tensor[:,1,:], 0)

    return truncated_tensor

In [ ]:
def estimation(unique_values, counts):

    # Estimation of the Fellegi-Sunter model using the Anandkumar et al. (2014)'s tensor decomposition approach

    # Inputs:
    # - unique_values: This three-dimensional tensor represents all the observed patterns of variables/modalities.
    #                  The first dimension represents the possible patterns.
    #                  The second dimension represents all the variables.
    #                  The third dimension represents all the possible modalities.
    # - counts: Vector which contains for each observed pattern, the number of observations.

    # Outputs:
    # - lambda_hat
    # - pi_hat

    n_features = unique_values.shape[1] # Number of variables

    n_modalities = unique_values.shape[2] # Number of possible modalities (largest across all variables)

    pi_hat = [] # Vector storing the estimated pi

    lambda_hat = [] # Vector storing the estimated lambda

    # Loop over all variables

    for feature in range(n_features):

        # Randomly sample two other variables

        dim1, dim2 = random.sample([i for i in range(n_features) if i != feature], k = 2)

        # Symmetrization

        truncated_tensor = symmetrization(unique_values, counts, dim1, dim2, feature)

        # Computation of the second and third moments

        M2 = tl.tenalg.batched_outer([truncated_tensor[:,0,:], truncated_tensor[:,1,:]])

        M3 = tl.tenalg.batched_outer([M2, truncated_tensor[:,2,:]])

        M2 = tl.tenalg.mode_dot(M2, counts, mode = 0) / np.sum(counts)

        M3 = tl.tenalg.mode_dot(M3, counts, mode = 0) / np.sum(counts)

        # Whitening of the third moment

        eigenvalues, eigenvectors = np.linalg.eig(tl.to_numpy(M2)) # Eigenvector decomposition

        eigenvalues = eigenvalues[0:2]

        eigenvectors = eigenvectors[:,0:2]

        W = tl.tensor(np.matmul(eigenvectors, np.diag(1 / np.sqrt(eigenvalues)))) # Whitening matrix

        M3 = tl.tenalg.multi_mode_dot(M3, [W, W, W], transpose = True)

        # Tensor decomposition via Symmetric CP Decomposition via Robust Symmetric Tensor Power Iteration

        weights, factors = tl.decomposition.symmetric_parafac_power_iteration(M3, rank = 2)

        # Unwhitening of the weights and factors (and appending to vector storing estimated parameters)

        U = scipy.linalg.pinv(tl.to_numpy(W).T) # Unwhitening matrix

        weights = tl.to_numpy(weights)

        factors = tl.to_numpy(factors)

        factors = np.matmul(U, factors)

        factors = np.multiply(weights, factors)

        weights = np.square(1 / weights)

        lambda_hat.append(weights)

        pi_hat.append(factors)

    return lambda_hat, pi_hat

In [ ]:
def match_probability_tensorized(gamma, pi, lmbda):

    # This function computes the estimated match probability for all patterns of similarity levels over variables.

    tensor_prob = []

    for m in range(2): # Loop over the latent state (i.e., match or no match)

        for k in range(K): # Loop over variables

            if k == 0:

                tensor_prob.append(np.log(tl.tenalg.mode_dot(gamma[:,k,:],
                                                             tl.transpose(tl.tensor(pi[k,:,m])), mode = 1)))

            else:

                tensor_prob[m] += np.log(tl.tenalg.mode_dot(gamma[:,k,:],
                                                            tl.transpose(tl.tensor(pi[k,:,m])), mode = 1))

        tensor_prob[m] = np.exp(tensor_prob[m])

    result = (lmbda * tensor_prob[1]) / ((1 - lmbda) * tensor_prob[0] + lmbda * tensor_prob[1])

    return np.reshape(tl.to_numpy(result), newshape = result.shape[0])

In [ ]:
def estimation_EM(unique_values, counts, convergence_condition = 1e-5):

    # Parameter Initialization

    lmbda = random.uniform(0, 1/2)

    pi_0 = np.random.dirichlet(np.ones(unique_values.shape[2]), unique_values.shape[1])

    pi_0 = -np.sort(-pi_0)

    pi_0 = np.transpose(pi_0)

    pi_1 = np.random.dirichlet(np.ones(unique_values.shape[2]), unique_values.shape[1])

    pi_1 = np.sort(pi_1)

    pi_1 = np.transpose(pi_1)

    # Proceed with E- and M-Steps Until Convergence

    convergence_status = False

    while not convergence_status:

        pi = np.stack((pi_0, pi_1), axis = -1)

        # E-Step: Compute Match Probability for every

        ksi = match_probability_tensorized(unique_values, tl.tensor(pi), lmbda)

        # M-Step

        lmbda = np.dot(ksi, counts) / sum(counts)

        pi_1 = tl.to_numpy(tl.sum(tl.tenalg.batched_outer((unique_values, tl.tensor(ksi * counts))), axis = 0)) / np.dot(ksi, counts)

        pi_1 = np.reshape(pi_1, newshape = (unique_values.shape[1], unique_values.shape[2]))

        pi_0 = tl.to_numpy(tl.sum(tl.tenalg.batched_outer((unique_values, tl.tensor((1 - ksi) * counts))), axis = 0)) / np.dot(1 - ksi, counts)

        pi_0 = np.reshape(pi_0, newshape = (unique_values.shape[1], unique_values.shape[2]))

        # Convergence is achieved when the largest varition in pi_kml is smaller than convergence_condition

        if np.max(np.absolute(pi - np.stack((pi_0, pi_1), axis = -1))) < convergence_condition:

            convergence_status = True

    return lmbda, np.stack((pi_0, pi_1), axis = -1)

In [ ]:
K = 3